In [3]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import f1_score

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 학습 모델 저장을 위한 라이브러리
import pickle
import joblib

### 프로젝트 셋팅

In [3]:
# 학습이 완료된 모델을 저장할 파일 이름
best_model_path = 'model/best_model_player_data.dat'

# 교차검증 횟수
cv_count = 10

# 교차 검증
kfold = KFold(n_splits=cv_count, shuffle=True, random_state=1)

# 평가 결과를 담을 리스트
# 필요하다면 다른 것도 만들어주세요
f1_score_list = []

# 학습 모델 이름
model_name_list = []

### 데이터 준비

In [4]:
pit_23 = pd.read_csv('23투수.csv')
pit_24 = pd.read_csv('24투수.csv')
sal_data_final = pd.read_csv('통합_연봉데이터.csv')

In [6]:
#display(pit_23.head(5))
#display(pit_24.head(5))
#display(sal_data_final.head(5))

In [7]:
# 1. 칼럼명 변경
pit_23 = pit_23.rename(columns={'Name_x': '선수'})
pit_24 = pit_24.rename(columns={'Name_x': '선수'})
#display(pit_23.head(5))

In [12]:
# 1. 연도 2023 데이터만 추출
sal_2023 = sal_data_final[sal_data_final['연도'] == 2023]

# 2. key 기준으로 일치 여부 확인 (선수, 팀, pid 모두 일치)
common_mask = (
    pit_23[['선수', '팀', 'pid']]
    .apply(tuple, axis=1)
    .isin(sal_2023[['선수', '팀', 'pid']].apply(tuple, axis=1))
)

# 3. 일치하는 데이터 중 연봉만 다른 경우 찾기
diff_rows = []
for idx in pit_23[common_mask].index:
    row = pit_23.loc[idx]
    matching = sal_2023[
        (sal_2023['선수'] == row['선수']) &
        (sal_2023['팀'] == row['팀']) &
        (sal_2023['pid'] == row['pid'])
    ]
    if not matching.empty and matching['연봉(만원)'].values[0] != row['연봉(만원)']:
        diff_rows.append({
            '선수': row['선수'],
            '팀': row['팀'],
            'pid': row['pid'],
            'pit_연봉': row['연봉(만원)'],
            'sal_연봉': matching['연봉(만원)'].values[0]
        })

# 4. 결과 확인
import pandas as pd
diff_df = pd.DataFrame(diff_rows)
display(diff_df)

""


In [7]:
sal_data_final

,선수,연봉(만원),WAR,WAR당 연봉,팀,pid,연봉,팀연봉순위,전체연봉순위,연도
0,채은성,180000.0,2.36,76221,한화 이글스,11215,180000.0,3.0,12.0,2023
1,최재훈,80000.0,3.90,20492,한화 이글스,10170,80000.0,9.0,94.0,2023
2,스미스,70000.0,0.09,756875,한화 이글스,15540,70000.0,10.0,113.0,2023
3,오그레디,70000.0,-0.87,-80365,한화 이글스,15541,70000.0,11.0,114.0,2023
4,이태양,66000.0,1.44,45702,한화 이글스,10609,66000.0,12.0,128.0,2023
...,...,...,...,...,...,...,...,...,...,...
2529,송지후,3000.0,-0.34,-8727,키움 히어로즈,16130,3000.0,243.0,2527.0,2025
2530,김주훈,3000.0,0.00,-,키움 히어로즈,16129,3000.0,244.0,2528.0,2025
2531,서유신,3000.0,-0.24,-12551,키움 히어로즈,15483,3000.0,245.0,2529.0,2025
2532,박성빈,3000.0,0.00,-,키움 히어로즈,15479,3000.0,246.0,2530.0,2025


In [8]:
select_cols = ['선수', '팀', 'pid', '연봉(만원)']
display(pit_23[select_cols])
display(sal_data_final.head(5))

,선수,팀,pid,연봉(만원)
0,양현종,KIA 타이거즈,10058,50000
1,진해수,LG 트윈스,10075,25000
2,고효준,SSG 랜더스,10124,8500
3,김광현,SSG 랜더스,10126,100000
4,박종훈,SSG 랜더스,10131,50000
...,...,...,...,...
292,산체스,한화 이글스,15643,40000
293,엘리아스,SSG 랜더스,15644,54000
294,맥키니,키움 히어로즈,15646,10000
295,산체스,KIA 타이거즈,15648,25000


,선수,연봉(만원),WAR,WAR당 연봉,팀,pid,연봉,팀연봉순위,전체연봉순위,연도
0,채은성,180000.0,2.36,76221,한화 이글스,11215,180000.0,3.0,12.0,2023
1,최재훈,80000.0,3.90,20492,한화 이글스,10170,80000.0,9.0,94.0,2023
2,스미스,70000.0,0.09,756875,한화 이글스,15540,70000.0,10.0,113.0,2023
3,오그레디,70000.0,-0.87,-80365,한화 이글스,15541,70000.0,11.0,114.0,2023
4,이태양,66000.0,1.44,45702,한화 이글스,10609,66000.0,12.0,128.0,2023


In [9]:
# 병합 전 기존 연봉을 임시 칼럼으로 저장
pit_23['연봉_기존'] = pit_23['연봉(만원)']

# 2023 연봉 데이터 준비
sal_2023 = sal_data_final[sal_data_final['연도'] == 2023]
sal_2023_key = sal_2023[['선수', '팀', 'pid', '연봉(만원)']].rename(columns={'연봉(만원)': '연봉_새로운'})

# 병합
pit_23 = pit_23.merge(sal_2023_key, on=['선수', '팀', 'pid'], how='left')

# 연봉이 다르게 바뀐 선수만 필터링
changed = pit_23[pit_23['연봉_기존'] != pit_23['연봉_새로운']]

print(f"연봉이 변경된 선수 수: {len(changed)}명")

연봉이 변경된 선수 수: 9명


In [10]:
# 비교를 위해 필요한 컬럼만 보기
display(changed[['선수', '팀', 'pid', '연봉_기존', '연봉_새로운']])

,선수,팀,pid,연봉_기존,연봉_새로운
44,김태훈,키움 히어로즈,10871,18000,NaN
45,김태훈,키움 히어로즈,10871,18000,NaN
74,심재민,KT 위즈,11311,8300,NaN
84,최원태,키움 히어로즈,11379,35000,NaN
85,채지선,LG 트윈스,11387,3800,NaN
86,채지선,LG 트윈스,11387,3800,NaN
271,김동규,LG 트윈스,15485,3000,NaN
272,김동규,LG 트윈스,15485,3000,NaN
288,와이드너,NC 다이노스,15536,59800,NaN


In [11]:
10/0

ZeroDivisionError: division by zero

In [ ]:
# 1. 2023년 데이터만 필터링
sal_2023 = sal_data_final[sal_data_final['연도'] == 2023]

# 2. 필요한 칼럼만 선택
sal_2023_key = sal_2023[['선수', '팀', 'pid', '연봉(만원)']]

# 3. pit_23과 key 기준 병합 (왼쪽 기준: pit_23)
pit_23 = pit_23.drop(columns='연봉(만원)', errors='ignore')  # 기존 연봉 칼럼 제거 (있다면)

pit_23 = pit_23.merge(
    sal_2023_key,
    on=['선수', '팀', 'pid'],
    how='left'  # pit_23에만 있는 선수도 유지
)

display(pit_23[select_cols])

In [ ]:
10/0

In [ ]:
# 데이터 준비
df1 = pd.read_csv('data/breast_cancer.csv')

# 입력과 결과로 나눈다.
X = df1.drop('target', axis=1)
y = df1['target']